In [ ]:
# libraries importation
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
from datetime import datetime
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import mutual_info_regression
from sklearn.cluster import KMeans

from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import holidays


In [ ]:
# Data insertion and initial processing
csv_path = "./nyc_taxi_data_2014.csv"
df = pd.read_csv(csv_path)

# Use a random sample for a faster development and testing cycle.
# Comment out this line to use the full dataset.
df = df.sample(n=200_000, random_state=42)
print(f"number of records: {len(df)}")

# Feature Selection & Creation
cols_to_use = [
    'pickup_datetime', 'dropoff_datetime', 'pickup_longitude', 'pickup_latitude',
    'dropoff_longitude', 'dropoff_latitude', 'passenger_count', 'vendor_id',
    'store_and_fwd_flag', 'tolls_amount'
]
df = df[cols_to_use].copy()

# Time processing
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'], errors='coerce')
df['dropoff_datetime'] = pd.to_datetime(df['dropoff_datetime'], errors='coerce')
df.dropna(subset=['pickup_datetime', 'dropoff_datetime'], inplace=True)

# Generated temporal features
df['pickup_hour'] = df['pickup_datetime'].dt.hour
df['day_of_week'] = df['pickup_datetime'].dt.weekday
df['month'] = df['pickup_datetime'].dt.month
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Trip duration
df['trip_duration_min'] = (df['dropoff_datetime'] - df['pickup_datetime']).dt.total_seconds() / 60
df = df[df['trip_duration_min'] > 0].copy()

# Function to calculate haversine distance on Earth's surface
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # km
    if pd.isna(lon1) or pd.isna(lat1) or pd.isna(lon2) or pd.isna(lat2):
        return np.nan
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

df['trip_distance_km'] = df.apply(
    lambda r: haversine(r['pickup_longitude'], r['pickup_latitude'],
                        r['dropoff_longitude'], r['dropoff_latitude']), axis=1)

# Average speed
df['avg_speed_kmph'] = df['trip_distance_km'] / (df['trip_duration_min'] / 60)

df = df.dropna(subset=['trip_distance_km', 'avg_speed_kmph'])

# Clustering pickup and dropoff locations
coords_pickup = df[['pickup_latitude', 'pickup_longitude']].dropna()
kmeans_pickup = KMeans(n_clusters=50, random_state=42, n_init=10)
pickup_clusters = kmeans_pickup.fit_predict(coords_pickup)
df.loc[coords_pickup.index, 'pickup_cluster'] = pickup_clusters

coords_dropoff = df[['dropoff_latitude', 'dropoff_longitude']].dropna()
kmeans_dropoff = KMeans(n_clusters=50, random_state=42, n_init=10)
dropoff_clusters = kmeans_dropoff.fit_predict(coords_dropoff)
df.loc[coords_dropoff.index, 'dropoff_cluster'] = dropoff_clusters

# Holiday detection
try:
    unique_years = df['pickup_datetime'].dt.year.unique()
    us_holidays = holidays.UnitedStates(years=unique_years)
    df['is_holiday'] = df['pickup_datetime'].dt.date.apply(lambda d: d in us_holidays).astype(int)
except Exception as e:
    print(f"Error detecting holidays: {e}")
    df['is_holiday'] = 0

# Bearing angle
def bearing(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    x = sin(dlon) * cos(lat2)
    y = cos(lat1)*sin(lat2) - sin(lat1)*cos(lat2)*cos(dlon)
    theta = atan2(x, y)
    bearing_deg = (np.degrees(theta) + 360) % 360
    return bearing_deg

df['bearing'] = df.apply(
    lambda r: bearing(r['pickup_latitude'], r['pickup_longitude'],
                      r['dropoff_latitude'], r['dropoff_longitude']), axis=1)

# Weather data
df_weather = pd.read_csv("weather_nyc_2014.csv")
df_weather['date'] = pd.to_datetime(df_weather['date'])
df['date'] = df['pickup_datetime'].dt.date
df_weather['date'] = df_weather['date'].dt.date

# Merge weather data with taxi data based on pickup date
df = df.merge(df_weather, on='date', how='left')
df.drop(columns=['date'], inplace=True)

# Composite weather feature
df['weather_impact'] = df['precip_mm'] * 2 + df['windgust_kph'] * 0.1 - df['avg_temp_c'] * 0.05

# Temporal-spatial interaction features
df['is_peak_hour'] = df['pickup_hour'].apply(lambda h: 1 if (7 <= h <= 9 or 16 <= h <= 19) else 0)
df['holiday_peak_flag'] = ((df['is_holiday'] == 1) & (df['is_peak_hour'] == 1)).astype(int)

# Trip count feature per cluster and hour
df['pickup_cluster_hour'] = df['pickup_cluster'].astype(str) + '_' + df['pickup_hour'].astype(str)
cluster_hour_counts = df.groupby('pickup_cluster_hour').size()
df['pickup_cluster_hour_trip_count'] = df['pickup_cluster_hour'].map(cluster_hour_counts).fillna(0)
df.drop(columns=['pickup_cluster_hour'], inplace=True)

df.fillna({'store_and_fwd_flag': 'U'}, inplace=True)

df = df.replace([np.inf, -np.inf], np.nan)
df = df.dropna()

print(f"Final number of records: {len(df)}")
print(df.head())

In [ ]:
# df.isnull().sum()

# df.columns

# df.hist(figsize=(20, 15))  
# plt.tight_layout()         
# plt.show()

In [ ]:
# --- Feature Selection and Data Splitting ---

# Define the target and feature columns for the model
target = 'trip_duration_min'
features = [
    'trip_distance_km',
    'vendor_id',
    'pickup_cluster',
    'dropoff_cluster',
    'pickup_cluster_hour_trip_count',
    'passenger_count',
    
    # Spatial Features
    'pickup_longitude',
    'pickup_latitude',
    'dropoff_longitude',
    'dropoff_latitude',
    'bearing',
    
    # Temporal Features
    'pickup_hour',
    'day_of_week',
    # 'month',  # Excluded based on correlation analysis
    'is_weekend',
    'is_peak_hour',
    'holiday_peak_flag',
    
    # Weather Features
    'weather_impact',
    'avg_temp_c',
    # 'precip_mm',  # Excluded based on low feature importance
    'windgust_kph',
    'condition',
]

# Separate features (X) and target (y)
X = df[features]
Y = df[target]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


# --- Data Encoding ---

# One-Hot Encoding
# This approach is good for tree-based models like XGBoost as it doesn't assume any ordinal relationship.
X_train_ohe = pd.get_dummies(X_train)
X_test_ohe = pd.get_dummies(X_test)

# Align columns to ensure the same features are in both train and test sets
X_train_ohe, X_test_ohe = X_train_ohe.align(X_test_ohe, join='left', axis=1, fill_value=0)


# Label Encoding
# This is an alternative, more memory-efficient approach for tree-based models.
cat_cols = X.select_dtypes(include='object').columns.tolist()
encoders = {}

X_train_le = X_train.copy()
X_test_le = X_test.copy()

for col in cat_cols:
    le = LabelEncoder()
    X_train_le[col] = le.fit_transform(X_train[col])
    X_test_le[col] = le.transform(X_test[col])
    encoders[col] = le

print("Shape of One-Hot Encoded data:", X_train_ohe.shape)
print("Shape of Label Encoded data:", X_train_le.shape)

In [ ]:
# --- XGBoost Regressor Model Training and Evaluation (Basic) ---

# Set the training and testing data
X_train = X_train_ohe
X_test = X_test_ohe

# Initialize and train the XGBoost model with default hyperparameters
model = XGBRegressor(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# --- Reporting the model's performance ---
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"R² Score: {r2:.2f}")

In [ ]:
# --- XGBoost Model with Cross-Validation ---

# Set the training and testing data
# We use the One-Hot Encoded data as it performs well with XGBoost
X_train = X_train_ohe
X_test = X_test_ohe

# Initialize the XGBoost model with a baseline number of estimators
model = XGBRegressor(n_estimators=100, random_state=42)

# Define the scoring metrics for cross-validation
# Note: Negative scores are used for metrics that should be minimized (MAE, MSE).
# cross_validate will automatically handle this by maximizing the score.
scoring = {
    'MAE': 'neg_mean_absolute_error',
    'MSE': 'neg_mean_squared_error',
    'R2': 'r2'
}

# Perform 5-fold cross-validation on the training data
cv_results = cross_validate(
    model,
    X_train_ohe, 
    y_train,
    cv=5,
    scoring=scoring,
    return_train_score=True,
    n_jobs=-1  # Use all available CPU cores for faster processing
)

# You can then print and analyze the results
# print("Cross-Validation Results:")
# for score_name, scores in cv_results.items():
#     print(f"{score_name}: {scores.mean():.4f} +/- {scores.std():.4f}")

In [ ]:
# --- Analyzing Cross-Validation Results ---

# Loop through each metric to print the results
for metric in scoring.keys():
    test_scores = cv_results[f'test_{metric}']
    train_scores = cv_results[f'train_{metric}']
    
    # Invert the score for metrics that were minimized (like MAE and MSE)
    if 'neg' in scoring[metric]:
        test_scores = -test_scores
        train_scores = -train_scores

    # Print the mean and standard deviation of the scores
    print(f"Train {metric}: {np.mean(train_scores):.3f} ± {np.std(train_scores):.3f}")
    print(f"Test  {metric}: {np.mean(test_scores):.3f} ± {np.std(test_scores):.3f}\n")

In [ ]:
# --- Final XGBoost Model Training and Evaluation ---

# Set the training and testing data
X_train = X_train_ohe
X_test = X_test_ohe



# Initialize the final model with the best parameters
model = XGBRegressor(
    learning_rate=0.05,
    n_estimators=250,
    max_depth=10,
    subsample=0.9,
    colsample_bytree=0.7,
    random_state=42,
    n_jobs=-1
)

# Train the model on the full training set
model.fit(X_train, y_train)pip install numpy==2.3.2


# Make predictions on the unseen test data
y_pred = model.predict(X_test)

# --- Reporting the Final Performance ---
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5 
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}")
print(f"MSE: {mse:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R² Score: {r2:.2f}")

In [ ]:
# --- Analyzing Feature Importance from the Model ---

# Get feature importances from the trained model
importances = model.feature_importances_

# Get the names of the features
feature_names = X_train.columns

# Create a DataFrame to hold feature names and their importance scores
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
}).sort_values(by='Importance', ascending=True)

# Plot the feature importance scores
plt.figure(figsize=(6, 4))
plt.barh(feature_importance_df['Feature'], feature_importance_df['Importance'], color='teal')
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.title("Feature Importance from XGBoost Model")
plt.grid(axis='x', linestyle='--', alpha=0.6)
plt.tight_layout() 
plt.show()

# Print the feature importance table
print("--- Feature Importance Table ---")
print(feature_importance_df.to_string())

In [ ]:
# --- Permutation Importance Analysis ---

# Calculate permutation importance on the test data
# This method measures how much the model's performance decreases
# when a single feature's values are randomly shuffled.
result = permutation_importance(
    model, 
    X_test, 
    y_test, 
    n_repeats=10,
    random_state=42,
    n_jobs=-1  # Use all available cores for faster processing
)

# --- Display Results ---

# Sort the features by their importance score
perm_sorted_idx = result.importances_mean.argsort()

perm_importance_df = pd.DataFrame({
    'Feature': X_test.columns[perm_sorted_idx],
    'Importance': result.importances_mean[perm_sorted_idx],
    'Std': result.importances_std[perm_sorted_idx]
})

print("\nPermutation Importance Results:")
print(perm_importance_df.sort_values(by='Importance', ascending=False).to_string())

In [ ]:
# --- Analyzing Model Errors ---

# Create a copy of the test data for error analysis
error_df = X_test.copy() 
error_df['actual_duration'] = y_test
error_df['predicted_duration'] = y_pred

# Calculate the error and absolute error
error_df['error'] = error_df['predicted_duration'] - error_df['actual_duration']
error_df['abs_error'] = np.abs(error_df['error'])

# Filter trips with error above a certain threshold (e.g., 20 minutes)
error_threshold = 20
high_error_trips = error_df[error_df['abs_error'] > error_threshold].copy()

# Sort by the highest error to see the worst predictions
high_error_trips = high_error_trips.sort_values(by='abs_error', ascending=False)

print(f"Number of trips with error greater than {error_threshold} minutes: {len(high_error_trips)} trips")

print("\nTrips with the highest error:")
print(high_error_trips.head())

print("\nAnalysis of high-error trips by pickup hour:")
print(high_error_trips['pickup_hour'].value_counts(normalize=True).head())

print("\nAnalysis of high-error trips by pickup cluster:")
print(high_error_trips['pickup_cluster'].value_counts(normalize=True).head())

print("\nAnalysis of high-error trips by dropoff cluster:")
print(high_error_trips['dropoff_cluster'].value_counts(normalize=True).head())

print("\nDescription of trip distance for high-error trips:")
print(high_error_trips['trip_distance_km'].describe())

print(f"Under-predicted duration (model was optimistic): {100 * (high_error_trips['error'] < 0).mean():.2f}%")
print(f"Over-predicted duration (model was pessimistic): {100 * (high_error_trips['error'] > 0).mean():.2f}%")

In [ ]:
# --- Statistical Analysis of Prediction Errors ---

# Calculate errors and absolute errors
errors = y_test - y_pred
abs_errors = np.abs(errors)

# Create a DataFrame to hold all error-related information
df_error = X_test.copy()
df_error['y_true'] = y_test
df_error['y_pred'] = y_pred
df_error['error'] = errors
df_error['abs_error'] = abs_errors

# Print key statistics of the errors
print("\nError statistics:")
print(df_error[['error', 'abs_error']].describe())

# 1. Visualize the error distribution
plt.figure(figsize=(10,4))
sns.histplot(df_error['error'], bins=50, kde=True, color='coral')
plt.title('Distribution of Prediction Errors')
plt.xlabel('Error (Predicted - Actual)')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# # Correlation Analysis and Visualization

# # The target variable for our analysis
# target_col = 'trip_duration_min'

# # Calculate the correlation matrix for all numeric features
# corr_matrix = df.corr(numeric_only=True)

# # Heatmap of feature correlations
# plt.figure(figsize=(12, 10))
# sns.heatmap(corr_matrix, cmap='coolwarm', annot=True, fmt=".2f")
# plt.title('Correlation Matrix')
# plt.show()

# # Correlation of each feature with the target variable
# target_corr = corr_matrix[target_col].drop(target_col).sort_values(ascending=False)

# plt.figure(figsize=(8, 6))
# sns.barplot(x=target_corr.values, y=target_corr.index, palette='viridis')
# plt.title(f'Correlation with Target: {target_col}')
# plt.xlabel('Correlation')
# plt.ylabel('Feature')
# plt.grid(True, axis='x', linestyle='--', alpha=0.5)
# plt.tight_layout()
# plt.show()

# # print("Correlation with target:")
# # print(target_corr)

# # print("\nFull correlation matrix:")
# # print(corr_matrix)

In [ ]:
# # Feature Analysis: Correlation & Mutual Information

# target_col = 'trip_duration_min'
# X = df.drop(columns=[target_col])
# y = df[target_col]

# # --- Pearson and Spearman Correlation Comparison ---
# num_df = df.select_dtypes(include=['number'])

# # Ensure the target column is included for correlation calculation
# if target_col not in num_df.columns:
#     num_df[target_col] = df[target_col]

# if target_col in num_df.columns:
#     pearson_corr = num_df.corr(method='pearson')[target_col].drop(target_col)
#     spearman_corr = num_df.corr(method='spearman')[target_col].drop(target_col)

#     comparison = pd.DataFrame({
#         'Pearson': pearson_corr,
#         'Spearman': spearman_corr,
#         'Diff': (pearson_corr - spearman_corr).abs()
#     }).sort_values('Diff', ascending=False)

#     print("--- Correlation Comparison with Target ---\n")
#     print(comparison.to_string())
#     print("\n-------------------------------------------")

# # --- Mutual Information ---
# datetime_cols = X.select_dtypes(include=['datetime64[ns]', 'datetime']).columns

# cols_to_keep = X.columns.difference(datetime_cols)
# X_subset_for_mi = X[cols_to_keep]
# X_encoded = pd.get_dummies(X_subset_for_mi, drop_first=True)

# mi_scores = mutual_info_regression(X_encoded, y, random_state=0)
# mi_series = pd.Series(mi_scores, index=X_encoded.columns).sort_values(ascending=False)

# # Mutual Information Feature Plot
# plt.figure(figsize=(8, 6))
# mi_series.head(20).plot(kind='barh', color='teal')
# plt.title('Top 20 MI Features with Target')
# plt.xlabel('Mutual Information')
# plt.gca().invert_yaxis()
# plt.grid(True, axis='x', linestyle='--', alpha=0.5)
# plt.tight_layout()
# plt.show()

In [ ]:
# Baseline Models

In [ ]:
# # --- K-Nearest Neighbors (KNN) Model Training and Evaluation ---

# # Set the training and testing data
# X_train = X_train_ohe
# X_test = X_test_ohe

# # Scaling the data is crucial for distance-based models like KNN
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Initialize and train the KNN model
# # Using n_neighbors=15 as an example hyperparameter
# knn_model = KNeighborsRegressor(n_neighbors=15)
# knn_model.fit(X_train_scaled, y_train)

# # Make predictions on the scaled test data
# y_pred = knn_model.predict(X_test_scaled)

# # --- Reporting the model's performance ---
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"MAE: {mae:.2f}")
# print(f"MSE: {mse:.2f}")
# print(f"R² Score: {r2:.2f}")

In [ ]:
# # --- Linear Regression Model Training and Evaluation ---

# # Set the training and testing data
# X_train = X_train_ohe
# X_test = X_test_ohe

# # Scaling the data is essential for linear models
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Initialize and train the Linear Regression model
# lr_model = LinearRegression()
# lr_model.fit(X_train_scaled, y_train)

# # Make predictions on the scaled test data
# y_pred = lr_model.predict(X_test_scaled)

# # --- Reporting the model's performance ---
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"MAE: {mae:.2f}")
# print(f"MSE: {mse:.2f}")
# print(f"R² Score: {r2:.2f}")

In [ ]:
# #SVR


# X_train = X_train_ohe
# X_test = X_test_ohe

# #scaling
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# svr_model = SVR()
# svr_model.fit(X_train_scaled, y_train)

# y_pred = svr_model.predict(X_test_scaled)

# #Report
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# rmse = mse ** 0.5 
# r2 = r2_score(y_test, y_pred)

# print(f"MAE: {mae:.2f}")
# print(f"MSE: {mse:.2f}")
# print(f"R² Score: {r2:.2f}")

In [ ]:
# # --- Decision Tree Regressor Model Training and Evaluation ---

# # Set the training and testing data
# # Decision Tree models are not sensitive to feature scaling,
# # so we can use the raw encoded data.
# X_train = X_train_ohe
# X_test = X_test_ohe

# # Initialize and train the Decision Tree model
# dt_model = DecisionTreeRegressor(random_state=42)
# dt_model.fit(X_train, y_train)

# # Make predictions on the test data
# y_pred = dt_model.predict(X_test)


# # --- Reporting the model's performance ---
# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"MAE: {mae:.2f}")
# print(f"MSE: {mse:.2f}")
# print(f"R² Score: {r2:.2f}")

In [ ]:
# # --- Random Forest Regressor Model Training and Evaluation ---

# # Set the training and testing data
# # Random Forest, like other tree-based models, does not require feature scaling.
# X_train = X_train_ohe
# X_test = X_test_ohe

# # Initialize and train the Random Forest model
# # Using n_estimators=100 as a starting point.
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train, y_train)

# # --- Reporting the model's performance ---
# # Make predictions on the test data
# y_pred = model.predict(X_test)

# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"MAE: {mae:.2f}")
# print(f"MSE: {mse:.2f}")
# print(f"R² Score: {r2:.2f}")

In [ ]:
# # --- Random Forest Regressor Model (with Scaled Data) ---

# # Set the training and testing data
# X_train = X_train_ohe
# X_test = X_test_ohe

# # Scale the data to test its effect on the model
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Initialize and train the Random Forest model
# model = RandomForestRegressor(n_estimators=100, random_state=42)
# model.fit(X_train_scaled, y_train)

# # --- Reporting the model's performance ---
# # Make predictions on the scaled test data
# y_pred = model.predict(X_test_scaled)

# mae = mean_absolute_error(y_test, y_pred)
# mse = mean_squared_error(y_test, y_pred)
# r2 = r2_score(y_test, y_pred)

# print(f"MAE: {mae:.2f}")
# print(f"MSE: {mse:.2f}")
# print(f"R² Score: {r2:.2f}")

In [ ]:
# Hyperparameter Tuning

In [ ]:
# # --- Hyperparameter Tuning with RandomizedSearchCV ---

# # Use the One-Hot Encoded data for XGBoost
# X_train = X_train_ohe
# X_test = X_test_ohe

# # Initialize XGBoost model
# xgb_model = XGBRegressor(random_state=42)

# # Define the hyperparameters to tune
# param_distributions = {
#     'n_estimators': [100, 200, 500],
#     'max_depth': [3, 5, 7, 10],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'subsample': [0.7, 0.8, 1.0],
#     'colsample_bytree': [0.7, 0.8, 1.0]
# }

# # Define the evaluation metrics
# scoring = {
#     'R2': 'r2',
#     'MAE': 'neg_mean_absolute_error',
#     'MSE': 'neg_mean_squared_error'
# }

# # Initialize RandomizedSearchCV
# random_search = RandomizedSearchCV(
#     estimator=xgb_model,
#     param_distributions=param_distributions,
#     n_iter=50,
#     cv=5,
#     scoring=scoring,
#     refit='MAE',
#     n_jobs=-1,  # Use all available cores for speed
#     random_state=42,
#     return_train_score=True
# )

# # Run the search on the training data
# random_search.fit(X_train, y_train)

# # --- Analyze the Results ---

# # Find and print the best parameters
# print("\nBest Parameters:")
# print(random_search.best_params_)

# # Analyze results and convert negative scores to positive
# results_df = pd.DataFrame(random_search.cv_results_)
# results_df['mean_test_MAE'] = -results_df['mean_test_MAE']
# results_df['mean_test_MSE'] = -results_df['mean_test_MSE']
# results_df = results_df.sort_values(by='rank_test_MAE')

# print("\nTop 5 parameter sets by MAE:")
# print(results_df[[ 'params', 'mean_test_MAE', 'mean_test_MSE', 'mean_test_R2' ]].head(5))

# # --- Final Evaluation on Test Set ---

# # Get the best model from the search
# best_model = random_search.best_estimator_
# y_pred = best_model.predict(X_test)

# final_mae = mean_absolute_error(y_test, y_pred)
# final_mse = mean_squared_error(y_test, y_pred)
# final_r2 = r2_score(y_test, y_pred)
# final_rmse = np.sqrt(final_mse)

# print("\n--- Final Evaluation on Test Set ---")
# print(f"MAE: {final_mae:.2f}")
# print(f"MSE: {final_mse:.2f}")
# print(f"R² : {final_r2:.2f}")
# print(f"RMSE: {final_rmse:.2f}")

In [ ]:
# # --- Grid Search based on best parameters from Randomized Search ---

# X_train = X_train_ohe
# X_test = X_test_ohe

# xgb_model = XGBRegressor(random_state=42)

# # Define the hyperparameter grid for fine-tuning
# param_grid = {
#     'n_estimators': [100, 200, 250],
#     'max_depth': [8, 10, 12],
#     'learning_rate': [0.05, 0.1, 0.15],
#     'subsample': [0.7, 0.8, 0.9],
#     'colsample_bytree': [0.7, 0.8, 0.9]
# }

# # Define the evaluation metrics
# scoring = {
#     'R2': 'r2',
#     'MAE': 'neg_mean_absolute_error',
#     'MSE': 'neg_mean_squared_error'
# }

# # Initialize GridSearchCV
# grid_search = GridSearchCV(
#     estimator=xgb_model,
#     param_grid=param_grid,
#     scoring=scoring,
#     refit='MAE',  # Refit the best model based on the MAE score
#     cv=5,
#     n_jobs=-1,
#     return_train_score=True
# )

# # Run the grid search
# grid_search.fit(X_train, y_train)

# # --- Analyze the Results ---

# # Print the best parameters based on MAE
# print("\nBest Parameters based on MAE:")
# print(grid_search.best_params_)

# # Get the best model from the search
# best_model = grid_search.best_estimator_

# # Make predictions on the final test set
# y_pred = best_model.predict(X_test)

# # Calculate final metrics
# final_mae = mean_absolute_error(y_test, y_pred)
# final_mse = mean_squared_error(y_test, y_pred)
# final_rmse = np.sqrt(final_mse)
# final_r2 = r2_score(y_test, y_pred)

# # Print final evaluation metrics on the test data
# print("\nFinal Evaluation on Test Set:")
# print(f"MAE : {final_mae:.2f}")
# print(f"MSE : {final_mse:.2f}")
# print(f"RMSE: {final_rmse:.2f}")
# print(f"R²  : {final_r2:.2f}")

# # --- Display Top Models from Search ---
# results_df = pd.DataFrame(grid_search.cv_results_)

# # Convert negative scores to positive
# results_df['mean_test_MAE'] = -results_df['mean_test_MAE']
# results_df['mean_test_MSE'] = -results_df['mean_test_MSE']

# # Sort results by the best metric (MAE)
# results_df = results_df.sort_values(by='rank_test_MAE')

# # Display the top 5 models
# print("\n--- Top 5 Models by MAE ---")
# print(results_df[[
#     'params', 
#     'mean_test_MAE', 
#     'mean_test_MSE',
#     'mean_test_R2'
# ]].head(5))